<a href="https://colab.research.google.com/github/muhammad-naeem-tahir/Assignments/blob/main/PIAIC129482-Final-Muhammad-Naeem-Tahir_flower_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [2]:
!gdown --id 1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj

Downloading...
From: https://drive.google.com/uc?id=1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj
To: /content/archive.zip
472MB [00:05, 80.3MB/s]


In [3]:
from zipfile import ZipFile
file_name = '/content/archive.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import RMSprop

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
data_path='/content/flowers/flowers'

In [7]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')

In [8]:
datagen.flow_from_directory(data_path)

Found 4323 images belonging to 5 classes.


In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.4) # set validation split

train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(256, 256),
    batch_size=48,
    color_mode='grayscale',
    shuffle=True,
    class_mode='sparse',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(256, 256),
    batch_size=48,
    color_mode='grayscale',
    shuffle=True,
    class_mode='sparse',
    subset='validation') # set as validation data


Found 2597 images belonging to 5 classes.
Found 1726 images belonging to 5 classes.


In [10]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 256x256 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 1,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.2),
    # The fourth convolution
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [11]:
model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

In [13]:
histor=model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples //48,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 48,
    epochs = 25)

Epoch 1/25
54/54 [==============================] - 29s 548ms/step - loss: 1.6440 - accuracy: 0.2279 - val_loss: 1.6069 - val_accuracy: 0.1792
Epoch 2/25
54/54 [==============================] - 29s 536ms/step - loss: 1.5796 - accuracy: 0.2597 - val_loss: 1.5403 - val_accuracy: 0.3119
Epoch 3/25
54/54 [==============================] - 29s 543ms/step - loss: 1.5233 - accuracy: 0.3370 - val_loss: 1.5037 - val_accuracy: 0.3589
Epoch 4/25
54/54 [==============================] - 29s 534ms/step - loss: 1.4743 - accuracy: 0.3664 - val_loss: 1.4503 - val_accuracy: 0.3750
Epoch 5/25
54/54 [==============================] - 29s 532ms/step - loss: 1.4089 - accuracy: 0.4064 - val_loss: 1.3617 - val_accuracy: 0.4321
Epoch 6/25
54/54 [==============================] - 29s 531ms/step - loss: 1.2822 - accuracy: 0.4692 - val_loss: 1.2846 - val_accuracy: 0.4786
Epoch 7/25
54/54 [==============================] - 28s 526ms/step - loss: 1.2366 - accuracy: 0.4998 - val_loss: 1.2379 - val_accuracy: 0.4994

KeyboardInterrupt: ignored

In [ ]:
model.save('content\\flowers\\my_model1.h5')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(histor.history['accuracy'])
plt.plot(histor.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(histor.history['loss'])
plt.plot(histor.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(validation_generator,verbose=0) #'sgd' instead of rmsprop 

In [ ]:
losses=pd.DataFrame(histor.history)
losses[['loss','val_loss']].plot()

In [ ]:
y_pred1 = model.predict(validation_generator)

In [ ]:
np.argmax(y_pred1, axis=1)[0:30]

In [ ]:
np.array(test_label[0:30]).astype('int32')

In [ ]:
plt.figure(figsize = (20,20))
for i in range(10):
    img = validation_generator[600*i][0]
    plt.subplot(1,5,i+1)
    plt.imshow(validation_generator, cmap='gray')
    plt.axis("off")
    plt.title(validation_generator[600*i][1])
plt.show()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(validation_generator,y_pred1))